In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter("ignore")

import geopandas as gpd
import numpy as np
import pandas as pd
from powergenome.generators import GeneratorClusters
from powergenome.GenX import reduce_time_domain, add_misc_gen_values
from powergenome.params import DATA_PATHS

from powergenome.external_data import (
    make_demand_response_profiles,
    make_generator_variability,
)

from powergenome.load_profiles import (
    make_load_curves, 
    add_load_growth, 
    make_final_load_curves, 
    make_distributed_gen_profiles,
)
from powergenome.external_data import make_demand_response_profiles
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    check_settings,
    reverse_dict_of_lists,
    remove_feb_29
)

from pathlib import Path
import itertools

from powergenome.generators import load_ipm_shapefile
from powergenome.GenX import (
    network_line_loss,
    network_max_reinforcement,
    network_reinforcement_cost,
)
from powergenome.transmission import (
    agg_transmission_constraints,
    transmission_line_distance,
)

pd.options.display.max_columns = 200

In [2]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt

In [3]:
single_region = 0
if single_region==0:
    run_folder = 'OEO_regional'
    settings_file = 'OEO_regional_settings.yml'
else:
    run_folder = 'OEO_national'
    settings_file = 'OEO_national_settings.yml'
scenario = "p6"
elec_only=0

In [4]:
#pudl_engine, pudl_out = init_pudl_connection()
cwd = Path.cwd()

settings_path = (
    cwd / run_folder / settings_file
)
settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
scenario_definitions = pd.read_csv(
    settings["input_folder"] / settings["scenario_definitions_fn"]
)
scenario_settings = build_scenario_settings(settings, scenario_definitions)

pudl_engine, pudl_out, pg_engine = init_pudl_connection(
    freq="AS",
    start_year=min(settings.get("data_years")),
    end_year=max(settings.get("data_years")),
)

check_settings(settings, pg_engine)


    *****************************
    The ATB technology "LandbasedWind_Class4" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <tech>_<tech_detail> string might be included, but it is best practice to
    include the full name in this format. Check your settings file.
        

    *****************************
    The ATB technology "OffShoreWind_Class10" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <tech>_<tech_detail> string might be included, but it is best practice to
    include the full name in this format. Check your settings file.
        

    *****************************
    The ATB technology "UtilityPV_Class1" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <te

In [5]:
all_periods = list(scenario_settings.keys())
start_year = all_periods[0]
file_prefix = str(settings_path).replace('.yml','_')
if elec_only==1:
    file_prefix += 'elec_'

In [6]:
new_gen = pd.DataFrame()
for year in all_periods[1:]: #new gen for start year is included in all_gen
    gc = GeneratorClusters(pudl_engine, pudl_out, pg_engine, scenario_settings[year][scenario])
    new_gen_year = gc.create_new_generators()
    new_gen_year.loc[:,'operating_year'] = year
    new_gen = pd.concat([new_gen, new_gen_year]) #create new generators for periods beyond the first
    

210.70000000000002  MW without lat/lon


No model tag values found for DR ('NoneType' object has no attribute 'items')
Transmission investment costs are missing or zero for some resources and will not be included in the total investment costs.
No model tag values found for DR ('NoneType' object has no attribute 'items')


210.70000000000002  MW without lat/lon


No model tag values found for DR ('NoneType' object has no attribute 'items')
Transmission investment costs are missing or zero for some resources and will not be included in the total investment costs.
No model tag values found for DR ('NoneType' object has no attribute 'items')


210.70000000000002  MW without lat/lon


...

No model tag values found for DR ('NoneType' object has no attribute 'items')
Transmission investment costs are missing or zero for some resources and will not be included in the total investment costs.
No model tag values found for DR ('NoneType' object has no attribute 'items')


210.70000000000002  MW without lat/lon


No model tag values found for DR ('NoneType' object has no attribute 'items')
Transmission investment costs are missing or zero for some resources and will not be included in the total investment costs.
No model tag values found for DR ('NoneType' object has no attribute 'items')


210.70000000000002  MW without lat/lon


No model tag values found for DR ('NoneType' object has no attribute 'items')
Transmission investment costs are missing or zero for some resources and will not be included in the total investment costs.
No model tag values found for DR ('NoneType' object has no attribute 'items')


In [7]:
if elec_only==1:
    load_curves = make_final_load_curves(pudl_engine, scenario_settings[start_year][scenario])
    elec_demand = load_curves.copy()
    for year in all_periods[1:]:
        load_curves_year = make_final_load_curves(pudl_engine, scenario_settings[year][scenario])
        load_curves_year.loc[:,'periods']=year
        elec_demand = pd.concat([elec_demand, load_curves_year])
else:
    load_curves = pd.read_csv('../oeo/data_aggregation/buildings/TablesForDB/EFS_hourly_DSD.csv')
    if single_region==1:
        load_curves = load_curves.drop(columns='Unnamed: 0').groupby(by=['LocalHourID', 'demand_name']).sum().reset_index().pivot_table(columns='demand_name', index='LocalHourID')['dds']
    else:
        load_curves['region_demand'] = load_curves['Region_OEO'] + '_' + load_curves['demand_name']
        load_curves = load_curves[['region_demand', 'LocalHourID', 'dds']]
        load_curves = load_curves.set_index('LocalHourID').pivot(columns='region_demand')['dds'].reset_index(drop=True)

In [8]:
gc = GeneratorClusters(pudl_engine, pudl_out, pg_engine, scenario_settings[start_year][scenario])
all_gens = gc.create_all_generators() #create existing and new generators for the first time period

#add misc_values from misc_gen_inputs_fn file in extra_inputs folder
all_gens = add_misc_gen_values(all_gens,settings)
new_gen = add_misc_gen_values(new_gen,settings)

gen_variability = make_generator_variability(all_gens)

210.70000000000002  MW without lat/lon


********************************
When adding plant entity/boiler info to generators and filling missing seasonal capacity values, technologyCoal Integrated Gasification Combined Cycle changed capacity from 779.0 to 965.5
********************************
********************************
When adding plant entity/boiler info to generators and filling missing seasonal capacity values, technologyConventional Hydroelectric changed capacity from 78253.6 to 78451.4
********************************
********************************
When adding plant entity/boiler info to generators and filling missing seasonal capacity values, technologyNatural Gas Fired Combined Cycle changed capacity from 284504.1 to 287637.1
********************************
********************************
When adding plant entity/boiler info to generators and filling missing seasonal capacity values, technologyNatural Gas Fired Combustion Turbine changed capacity from 139964.3 to 140514.5
********************************
***

Creating gdf
['Batteries', 'Onshore Wind Turbine', 'Solar Photovoltaic', 'Natural Gas Fired Combustion Turbine', 'Other_peaker', 'Natural Gas Fired Combined Cycle', 'Natural Gas with Compressed Air Storage', 'Biomass', 'Natural Gas Steam Turbine', 'Other Natural Gas', 'Conventional Hydroelectric', 'Nuclear', 'Geothermal', 'All Other']
Creating gdf
['Onshore Wind Turbine', 'Solar Photovoltaic', 'Natural Gas Fired Combined Cycle', 'Natural Gas Fired Combustion Turbine', 'Batteries', 'Conventional Hydroelectric', 'Other_peaker', 'All Other', 'Other Gases', 'Petroleum Coke', 'Conventional Steam Coal', 'Flywheels', 'Biomass', 'Natural Gas Steam Turbine', 'Other Natural Gas', 'Hydroelectric Pumped Storage', 'Geothermal']


No model tag values found for DR ('NoneType' object has no attribute 'items')
No model tag values found for DR ('NoneType' object has no attribute 'items')
Transmission investment costs are missing or zero for some resources and will not be included in the total investment costs.
No model tag values found for DR ('NoneType' object has no attribute 'items')


In [9]:
(
    reduced_resource_profile,
    reduced_load_profile,
    time_series_mapping,
    representative_point,
) = reduce_time_domain(gen_variability, load_curves, scenario_settings[start_year][scenario])

In [10]:
if len(settings['region_aggregations'])>1:
    transmission = agg_transmission_constraints(pg_engine=pg_engine, settings=settings)
    model_regions_gdf = load_ipm_shapefile(settings)
    transmission = transmission_line_distance(
        trans_constraints_df=transmission,
        ipm_shapefile=model_regions_gdf,
        settings=settings,
    )
    transmission = network_line_loss(transmission=transmission, settings=settings)
    transmission = network_reinforcement_cost(transmission=transmission, settings=settings)
    transmission = network_max_reinforcement(transmission=transmission, settings=settings)
    transmission.to_csv(file_prefix + 'transmission.csv', index=False)

In [11]:
gen_variability.to_csv(file_prefix + 'gen_variability.csv', index=False)
load_curves.to_csv(file_prefix + 'load_curves.csv', index=False)
reduced_load_profile.to_csv(file_prefix + 'reduced_load_profile_2P_2D.csv', index=False)
time_series_mapping.to_csv(file_prefix + 'time_series_mapping_2P_2D.csv', index=False)
reduced_resource_profile.to_csv(file_prefix + 'reduced_resource_2P_2D.csv', index=False)
all_gens.to_csv(file_prefix + 'all_gens.csv', index=False)
new_gen.to_csv(file_prefix + 'new_gen.csv', index=False)

In [12]:
#additional time-domain combinations of periods and days per period.
for P,D in [[3,4],[4,6],[6,8], [3,7], [2,7]]:
    scenario_settings[start_year][scenario]['time_domain_days_per_period'] = D
    scenario_settings[start_year][scenario]['time_domain_periods'] = P
    (
        reduced_resource_profile,
        reduced_load_profile,
        time_series_mapping,
        representative_point
    ) = reduce_time_domain(gen_variability, load_curves, scenario_settings[start_year][scenario])
    reduced_load_profile.to_csv(file_prefix + 'reduced_load_profile_' + str(P) + 'P_' + str(D) + 'D.csv', index=False)
    time_series_mapping.to_csv(file_prefix + 'time_series_mapping_' + str(P) + 'P_' + str(D) + 'D.csv', index=False)
    reduced_resource_profile.to_csv(file_prefix + 'reduced_resource_' + str(P) + 'P_' + str(D) + 'D.csv', index=False)

In [13]:
gen_variability_zephyr = pd.read_csv('../../zephyr-main/zephyr_wind_solar_cfs.csv',index_col=0)

In [14]:
#create zephyr files
for P,D in [[2,2],[3,4],[4,6],[6,8], [3,7], [2,7]]:
    scenario_settings[start_year][scenario]['time_domain_days_per_period'] = D
    scenario_settings[start_year][scenario]['time_domain_periods'] = P
    (
        reduced_resource_profile,
        reduced_load_profile,
        time_series_mapping,
        representative_point
    ) = reduce_time_domain(gen_variability_zephyr, load_curves, scenario_settings[start_year][scenario])
    reduced_load_profile.to_csv(file_prefix + 'zephyr_reduced_load_profile_' + str(P) + 'P_' + str(D) + 'D.csv', index=False)
    time_series_mapping.to_csv(file_prefix + 'zephyr_time_series_mapping_' + str(P) + 'P_' + str(D) + 'D.csv', index=False)
    reduced_resource_profile.to_csv(file_prefix + 'zephyr_reduced_resource_' + str(P) + 'P_' + str(D) + 'D.csv', index=False)